In [85]:
import pandas as pd
import pickle
import re
import nltk
import time
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from stop_words import get_stop_words
import numpy as np
from sklearn.cluster import KMeans


pd.set_option('display.max_colwidth', -1)

# 1-Loading and Data Transformation

In [86]:
df1=pd.read_csv("Transition_ecologique.csv")
df2=pd.read_csv("LA_TRANSITION_ECOLOGIQUE_Part1.csv")
df3=df=pd.read_csv("LA_TRANSITION_ECOLOGIQUE_Part2.csv")

df1.drop('id',axis=1,inplace=True)
df1=pd.melt(df1, id_vars=["reference", "title", "createdAt", "publishedAt", "updatedAt",
       "trashed", "trashedStatus", "authorId", "authorType", "authorZipCode"], 
                  var_name="Question", value_name="Value")
df2=pd.melt(df2, id_vars=["reference", "title", "createdAt", "publishedAt", "updatedAt",
       "trashed", "trashedStatus", "authorId", "authorType", "authorZipCode"], 
                  var_name="Question", value_name="Value")
df3=pd.melt(df3, id_vars=["reference", "title", "createdAt", "publishedAt", "updatedAt",
       "trashed", "trashedStatus", "authorId", "authorType", "authorZipCode"], 
                  var_name="Question", value_name="Value")

df1['Question']=df1.Question.str.split(' - ',-1).apply(lambda x: x[1])
df=pd.concat([df1,df2,df3])
df=df.loc[(df.trashed==False)]
df=df.loc[(df.title.isna()==False)]

Id=1
df['QuestionId']=0
for Question in df.Question.unique():
    df.loc[df.Question==Question,'QuestionId']=Id
    Id+=1
    
#Initialize Stop Words

StopWordsFr=stopwords.words("french")+['France','Ecologie','Environement','Stop']

/home/amine/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 2-Cleaning for TF-IDF

In [97]:
#Delete all special characters and Stop Words

def clean_Title(RawTitle):
    """
    Function to clean bill text to keep only letters and remove stopwords
    Returns a string of the cleaned bill text
    """
    letters_only = re.sub('[^a-zA-ZéèçàÔ]', ' ', RawTitle)
    words = letters_only.lower().split()
    StopWordsFr=stopwords.words("french")+['France','Ecologie','Environement','Stop']
    useful_words = [x for x in words if not x in StopWordsFr]
    # Combine words into a paragraph again
    useful_words_string = ' '.join(useful_words)
    return(useful_words_string)

Title_text_list_clean = map(clean_Title, df.title)
Clean_Titles_dictionary = dict(zip(df.title, Title_text_list_clean))

with open('Clean_Titles_dictionary.pickle', 'wb') as handle:
    pickle.dump(Clean_Titles_dictionary, handle)

In [98]:
Clean_Titles_dictionary

{'transition écologique': 'transition écologique',
 'La surpopulation': 'surpopulation',
 'climat': 'climat',
 'POLLUTION AIR EAU': 'pollution air eau',
 'Economie vs Ecologie': 'economie vs ecologie',
 'égalité territoriale de traitement': 'égalité territoriale traitement',
 'Nous sommes les gardiens de la terre et des passeurs pour nos enfants': 'les gardiens terre passeurs enfants',
 'Pollution de la planete': 'pollution planete',
 "imposer une écotaxe aux compagnies maritimes et d'aviation": 'imposer écotaxe compagnies maritimes aviation',
 'Ferroutage': 'ferroutage',
 'Ecologie participative et Intelligente': 'ecologie participative intelligente',
 'Transport': 'transport',
 'environnement': 'environnement',
 'danieln': 'danieln',
 'Pollution sans impôt supplémentaire': 'pollution sans imp supplémentaire',
 'ECOLOGIE': 'ecologie',
 'FIN DE LA CROISSANCE': 'fin croissance',
 'Semences de variétés traditionnelles': 'semences variétés traditionnelles',
 "n'oublions pas les campagnes"

In [89]:
#Tokenize words then apply TF-IDF

stemmer = PorterStemmer()

def stem_words(words_list, stemmer):
    return [stemmer.stem(word) for word in words_list]

def tokenize(text):
    tokens = nltk.word_tokenize(text,language='french')
    stems = stem_words(tokens, stemmer)
    return stems

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=StopWordsFr)
tfs = tfidf.fit_transform(Clean_Titles_dictionary.values())

# 3-Unsupervised K means clustering with Tf-idf Vectors

In [90]:
#K means

k = 100
km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=20,
                verbose=2)
km.fit(tfs)
cluster_assignments_dict = {}

for i in set(km.labels_):
    #print i
    current_cluster_Titles = [list(Clean_Titles_dictionary.keys())[x] for x in np.where(km.labels_ == i)[0]]
    cluster_assignments_dict[i] = current_cluster_Titles

Initialization complete
Iteration  0, inertia 51645.332
Iteration  1, inertia 48728.489
Iteration  2, inertia 47366.469
Iteration  3, inertia 46898.783
Iteration  4, inertia 46630.486
Iteration  5, inertia 46494.038
Iteration  6, inertia 46453.234
Iteration  7, inertia 46432.685
Iteration  8, inertia 46418.218
Iteration  9, inertia 46413.393
Iteration 10, inertia 46408.825
Iteration 11, inertia 46401.289
Iteration 12, inertia 46398.236
Iteration 13, inertia 46395.926
Iteration 14, inertia 46388.806
Iteration 15, inertia 46379.533
Iteration 16, inertia 46377.546
Iteration 17, inertia 46376.896
Iteration 18, inertia 46376.666
Iteration 19, inertia 46376.196
Iteration 20, inertia 46375.619
Iteration 21, inertia 46374.604
Iteration 22, inertia 46370.634
Iteration 23, inertia 46363.232
Iteration 24, inertia 46354.172
Iteration 25, inertia 46348.467
Iteration 26, inertia 46347.442
Iteration 27, inertia 46347.277
Iteration 28, inertia 46347.204
Iteration 29, inertia 46347.174
Iteration 30, in

Iteration  8, inertia 45439.237
Iteration  9, inertia 45430.638
Iteration 10, inertia 45417.793
Iteration 11, inertia 45406.440
Iteration 12, inertia 45392.214
Iteration 13, inertia 45381.871
Iteration 14, inertia 45373.856
Iteration 15, inertia 45360.473
Iteration 16, inertia 45349.946
Iteration 17, inertia 45347.526
Iteration 18, inertia 45347.208
Iteration 19, inertia 45347.021
Iteration 20, inertia 45346.882
Iteration 21, inertia 45346.558
Iteration 22, inertia 45346.241
Iteration 23, inertia 45345.712
Iteration 24, inertia 45345.393
Iteration 25, inertia 45344.843
Iteration 26, inertia 45343.631
Iteration 27, inertia 45342.631
Iteration 28, inertia 45341.478
Iteration 29, inertia 45339.267
Iteration 30, inertia 45338.731
Iteration 31, inertia 45338.726
Converged at iteration 31: center shift 0.000000e+00 within tolerance 7.240970e-09
Initialization complete
Iteration  0, inertia 79269.419
Iteration  1, inertia 46946.925
Iteration  2, inertia 46161.385
Iteration  3, inertia 45776.0

Iteration 24, inertia 45241.878
Converged at iteration 24: center shift 0.000000e+00 within tolerance 7.240970e-09
Initialization complete
Iteration  0, inertia 79156.516
Iteration  1, inertia 46816.464
Iteration  2, inertia 46153.550
Iteration  3, inertia 45784.143
Iteration  4, inertia 45607.304
Iteration  5, inertia 45499.692
Iteration  6, inertia 45446.241
Iteration  7, inertia 45418.278
Iteration  8, inertia 45361.392
Iteration  9, inertia 45328.508
Iteration 10, inertia 45311.425
Iteration 11, inertia 45295.957
Iteration 12, inertia 45270.336
Iteration 13, inertia 45259.613
Iteration 14, inertia 45258.713
Iteration 15, inertia 45258.382
Iteration 16, inertia 45257.918
Iteration 17, inertia 45257.555
Iteration 18, inertia 45257.377
Iteration 19, inertia 45257.241
Iteration 20, inertia 45257.005
Iteration 21, inertia 45256.517
Iteration 22, inertia 45255.697
Iteration 23, inertia 45255.239
Iteration 24, inertia 45254.912
Iteration 25, inertia 45254.696
Iteration 26, inertia 45254.2

In [91]:
cluster_themes_dict = {}

for key in cluster_assignments_dict.keys():
    current_tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=stopwords.words("french"))
    current_tfs = current_tfidf.fit_transform(map(clean_Title, cluster_assignments_dict[key]))
    
    current_tf_idfs = dict(zip(current_tfidf.get_feature_names(), current_tfidf.idf_))
    tf_idfs_tuples = current_tf_idfs.items()
    cluster_themes_dict[key] = sorted(tf_idfs_tuples, key = lambda x: x[1])[:5]

In [80]:
cluster_themes_dict[2]

[('velopp', 1.0),
 ('transport', 2.5769147207285403),
 ('hydrog', 3.4604156297797046),
 ('recherch', 3.67552700939665),
 ('cologiqu', 3.7589086183357012)]

In [ ]:
# Load Facebooks' pre-trained model.
model = gensim.models.KeyedVectors.load_word2vec_format('/home/amine/cc.fr.300.vec.1')  

In [ ]:
def average_Title(Title,model):
    Sentence = re.sub('[^a-zA-Z]', ' ', Title).lower().split(' ')
    Length=0
    avg=np.zeros((300,))
    for word in Sentence:
        if word not in get_stop_words('french') and word in model.vocab :
            avg+=model[word]
            Length+=1
        if Length>0:
            avg=avg/Length
    return avg

In [ ]:
Vectorized=[]
for title in df.title.unique():
        Vectorized_Title_Temp=average_Title(title,model)
        Vectorized.append(Vectorized_Title_Temp)
Vectorized_Titles_dictionary = dict(zip(df.title.unique(), Vectorized))

In [ ]:
from sklearn.cluster import KMeans

k = 100
km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=20,
                verbose=2)
km.fit_transform(Vectorized)
cluster_assignments_dict = {}

for i in set(km.labels_):
    #print i
    current_cluster_Titles = [list(Vectorized_Titles_dictionary.keys())[x] for x in np.where(km.labels_ == i)[0]]
    cluster_assignments_dict[i] = current_cluster_Titles
    
with open('Cluster_assignment.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
cluster_assignments_dict[89]

In [ ]:
print(km.labels_)

In [ ]:
len(df.loc[(df.trashed==False)].authorId.unique())

In [ ]:
df.title.isna()

In [ ]:
len(df.loc[(df.trashed==True)].authorId.unique())

In [ ]:
len(Vectorized)

In [66]:
tfs.shape

(64605, 12071)